# 0.0 Imports Dependencies

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification # conver string to sequence of numbers | 
import torch # extract sequence results 
import requests # requests from sites 
from bs4 import BeautifulSoup # extract data 
import re # creage regex functions

import pandas as pd # structure data
import numpy as np # data transformation 

# 1.0. Instantiate Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

Downloading: 100%|██████████| 39.0/39.0 [00:00<00:00, 11.6kB/s]
Downloading: 100%|██████████| 953/953 [00:00<00:00, 622kB/s]
Downloading: 100%|██████████| 851k/851k [00:00<00:00, 1.13MB/s] 
Downloading: 100%|██████████| 112/112 [00:00<00:00, 72.3kB/s]
Downloading: 100%|██████████| 638M/638M [00:32<00:00, 20.7MB/s] 


# 2.0. Encode and Calculate Sentiment

In [42]:
tokens = tokenizer.encode("more or las!", return_tensors="pt") # pt = pytorch

In [43]:
result = model(tokens)

In [44]:
result.logits

tensor([[ 0.0411,  0.0613,  0.0706, -0.2429,  0.0043]],
       grad_fn=<AddmmBackward0>)

In [41]:
int(torch.argmax(result.logits))+1 # these numbers represents the sentiment rating from 1 to 5 (higher the better)

5

# 3.0. Collect Reviews

In [45]:
r = requests.get("https://www.yelp.com/biz/shake-shack-orlando-8?osq=Shake+Shack")
soup = BeautifulSoup(r.text, "html.parser")
regex = re.compile(".*comment.*") 
results = soup.find_all("p", {"class":regex}) # get everything in the class "comment to claissfy"  p = paragraph
reviews = [result.text for result in results] #get only the reviewws, removing the tags

In [53]:
reviews[2]

"Hi Meagan G., thank you so much for the feedback! We appreciate your patronage as a customer and we'd love to have you back.Could you message us with your email address so we can grab your details?Thank you again,Jake"

# 4.0. Load Reviews into DataFrame and Score

In [54]:
df = pd.DataFrame(np.array(reviews), columns=["review"])

In [57]:
df["review"].iloc[0]

'First time eating here. Visiting from Va the food was delicious and the workers were friendly. The chicken sandwich was moist and filling also loved the mango Paradise drink.'

In [58]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors="pt")
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [62]:
sentiment_score(df["review"].iloc[12])

4

In [70]:
df["sentiment"] = df["review"].apply(lambda x: sentiment_score(x[:512]))

In [71]:
df

,review,sentiment
0,First time eating here. Visiting from Va the f...,5
1,Friday nights are super busy. Recommend to pla...,5
2,"Hi Meagan G., thank you so much for the feedba...",5
3,I got a bacon burger with fries. They put way ...,4
4,"Thank you for your feedback, Natalie. It's dis...",2
5,Overall this is a solid Shake Shack location. ...,4
6,"Hi Matthew,Thank you for your feedback. It's d...",2
7,Shake shack never misses. When you need a soli...,5
8,"Hi Andrew,Thanks for taking the time to leave ...",5
9,When I used to live in NY there were a handful...,5
